In [1]:
import json
from pathlib import Path
import sys
from functools import partial
from tqdm import tqdm_notebook as tqdm
sys.path.append("../") 

import html2text
from bs4 import BeautifulSoup

In [2]:
from src.data.utils import read_jsonl, extract_text_by_headers_html, dedup_results, html2text_parser, heuristic_clean_html

json_objs = read_jsonl("../data/raw/task1/crag_task_1_dev_v3_release.jsonl", 500)
json_objs_split1 = [json_obj for json_obj in json_objs if json_obj["split"] == 1]
json_objs = [json_obj for json_obj in json_objs if json_obj["split"] == 0]
# preprocessing_fn = partial(extract_text_by_headers_html, version="v2", split_sentences=False)
preprocessing_fn = html2text_parser
preprocessing_fn = partial(html2text_parser, version="v5")

In [26]:
i = 0
j = 0
json_obj = json_objs[i]["search_results"][j]
page_html = json_obj["page_result"]
page_name = json_obj["page_name"]
page_url = json_obj["page_url"]
page_html_cleaned = heuristic_clean_html(page_html, 0.9)
soup = BeautifulSoup(page_html, 'html.parser')


print(json_objs[i]["query"])
def traverse(node):
    stack = [node]
    segments = []
    html_segments = []
    while stack:
        current_node = stack.pop()
        if current_node.name is not None:
            if current_node.text == "References":
                break
            # Check if the node meets your criteria (e.g., has enough text)
            text_length = len(current_node.get_text(strip=True))
            if text_length < 2000 or len(current_node.contents) == 0:  # Example criteria
                # print(f"Skipping children of node: {current_node.name}, Text: {current_node.get_text(strip=True)}")
                if current_node.text.strip():
                    # print(h.handle(str(current_node)))
                    h = html2text.HTML2Text()
                    h.ignore_links = True
                    h.ignore_pictures = True
                    h.ignore_emphasis = True
                    h.ignore_images = True
                    html_segments.append(str(current_node))
                    segments.append(h.handle(str(current_node)).strip())
                    # segments.append(current_node.text.strip())
                continue
            # Add children to the stack in reverse order to maintain original order
            for child in reversed(current_node.contents):
                stack.append(child)
        else:
            if current_node.strip() and "div" not in current_node:
                segments.append(current_node.strip())
    return segments

def merge_small_segments(segments):
    merged_segments = []
    buffer = ""

    for segment in segments:
        if len(segment.strip()) < 100:
            buffer += segment.strip() + "\n"
        else:
            if buffer:
                merged_segments.append(buffer.strip() + "\n" + segment)
                buffer = ""
            else:
                merged_segments.append(segment)

    if buffer:
        merged_segments.append(buffer.strip())

    return merged_segments

html_segments, segments = traverse(page_html_cleaned)


how many 3-point attempts did steve nash average per game in seasons he made the 50-40-90 club?


In [27]:
print(page_url)
for segment in segments:
    print(segment)
    print("="*100)

https://www.basketball-reference.com/players/n/nashst01.html
* Sports Reference ®
  * Baseball
  * Football (college)
  * Basketball (college)
  * Hockey
  * Calcio
  * Blog
  * Stathead ®
  * Immaculate Grid
  * Questions or Comments?
  * Welcome  · Your Account
  * Logout
  * Ad-Free Login
  * Create Account

MENU

  * Players
  * Teams
  * Seasons
  * Leaders
  * Scores
  * WNBA
  * Draft
  * Stathead
  * Newsletter
  * Full Site Menu Below

You are here:

BBR Home Page > Players > N > Steve Nash

  * Welcome  · Your Account
  * Logout
  * Ad-Free Login
  * Create Account
#  Steve Nash

Stephen John Nash ▪ Twitter: SteveNash

(MVSteve, Two Time, Nashty)

Position:  Point Guard ▪  Shoots:  Right

6-3, 195lb (190cm, 88kg)

Born:  February 7, 1974 in Johannesburg, South Africa za

College:  Santa Clara

High School:  Saint Michaels University School in Victoria, Canada

Draft:  Phoenix Suns, 1st round (15th pick, 15th overall), 1996 NBA Draft

NBA Debut: November 1, 1996

Hall of Fame:

In [28]:
merged_segments = merge_small_segments(segments)
for merged_segment in merged_segments:
    print(merged_segment)
    print("="*100)

* Sports Reference ®
  * Baseball
  * Football (college)
  * Basketball (college)
  * Hockey
  * Calcio
  * Blog
  * Stathead ®
  * Immaculate Grid
  * Questions or Comments?
  * Welcome  · Your Account
  * Logout
  * Ad-Free Login
  * Create Account

MENU

  * Players
  * Teams
  * Seasons
  * Leaders
  * Scores
  * WNBA
  * Draft
  * Stathead
  * Newsletter
  * Full Site Menu Below

You are here:

BBR Home Page > Players > N > Steve Nash

  * Welcome  · Your Account
  * Logout
  * Ad-Free Login
  * Create Account
#  Steve Nash

Stephen John Nash ▪ Twitter: SteveNash

(MVSteve, Two Time, Nashty)

Position:  Point Guard ▪  Shoots:  Right

6-3, 195lb (190cm, 88kg)

Born:  February 7, 1974 in Johannesburg, South Africa za

College:  Santa Clara

High School:  Saint Michaels University School in Victoria, Canada

Draft:  Phoenix Suns, 1st round (15th pick, 15th overall), 1996 NBA Draft

NBA Debut: November 1, 1996

Hall of Fame: Inducted as Player in 2018 (Full List)

Career Length: 18 ye